In [10]:
# Лабораторная работа 1 по дисциплине МРЗвИС
# Выполнена студентом группы 121702
# БГУИР Заломов Роман Андреевич
#
# Вариант 15: Реализовать модель линейной рециркуляционной сети 
# с постоянным коэффициентом обучения и нормированными весовыми коэффициентами.
#
# Данный файл содержит алгоритмы построения графиков различных зависимостей

In [11]:
from lrnn import (
    LRNN,
    image_to_blocks,
    compress_image,
    decompress_image
)
import numpy as np
from PIL import Image
import plotly.graph_objects as go

In [12]:
MAX_RGB_VALUE = 255
COLOR_CHANNELS_AMOUNT = 3

In [13]:
# Collecting everything

block_width = 10
block_height = 10

n = block_height * block_width
# Hidden layer neuron amount
p = 20 

img = Image.open('car.jpg')
img_array = np.asarray(img)
shape = img_array.shape
blocks = image_to_blocks(img_array, block_height, block_width, overlap=0)

l = len(blocks)
# Compression coeff
print('Z =', (n*l) / ((n+l) * p+2))

color_df = ((2 * blocks / MAX_RGB_VALUE) - 1).reshape(len(blocks), -1, 3).transpose(0, 2, 1).reshape(-1, block_height * block_width)
train = np.matrix(color_df[np.random.choice(color_df.shape[0], int(color_df.shape[0] * 0.1))])


network = LRNN(n, p, 0.0015)
network.train(train, 15000, learn_by_loss=True, max_loss=2000)

compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)

compression_info_size = (
    compressed.size * compressed.itemsize +
    network.W_dec.size * network.W_dec.itemsize +
    np.array(shape).size * np.array(shape).itemsize +
    np.array((block_height, block_width)).size * np.array((block_height, block_width)).itemsize 
) * 8
print(f'Compression coefficient: {(img_array.size * img_array.itemsize * 8) / compression_info_size}')
print('Z =', (n*l) / ((n+l) * p + 2))

dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
dimg_array = np.asarray(dimg)
dimg.save('compression-decompression_test.jpg')

Z = 4.807507403561401
Epoch 1/15000, Loss: 2066.774966156342
Epoch 2/15000, Loss: 1923.2415613971286
Compression coefficient: 1.2334714821393329
Z = 4.807507403561401


## Relation between learning rate and epochs 

In [14]:
alphas = np.linspace(0.0001, 0.003, endpoint=True, num=10)
epochs_amount_mean: list[int] = []
fig = go.Figure()
for alpha in alphas:
    print(alpha)
    dl = LRNN(n, 20, alpha)
    dl.train(train, 15000, learn_by_loss=True, max_loss=1200)
    epochs_amount_mean.append(dl.epoch)
fig.add_trace(go.Scatter(x=alphas, y=epochs_amount_mean, mode='lines+markers',
                         name='Среднее значение'))
fig.update_layout(title='Зависимость количества итераций обучения от коэффициента обучения',
                  xaxis_title='Коэффициент обучения',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

0.0001
Epoch 1/15000, Loss: 3214.4550968476847
Epoch 2/15000, Loss: 2952.4839402198195
Epoch 3/15000, Loss: 2822.8659193335498
Epoch 4/15000, Loss: 2727.752846070586
Epoch 5/15000, Loss: 2652.326527628158
Epoch 6/15000, Loss: 2590.0545222116257
Epoch 7/15000, Loss: 2537.1971383545324
Epoch 8/15000, Loss: 2491.385067505622
Epoch 9/15000, Loss: 2451.0177507361927
Epoch 10/15000, Loss: 2414.962753664626
Epoch 11/15000, Loss: 2382.39010079216
Epoch 12/15000, Loss: 2352.672912052774
Epoch 13/15000, Loss: 2325.326835082033
Epoch 14/15000, Loss: 2299.971289225926
Epoch 15/15000, Loss: 2276.3016922955794
Epoch 16/15000, Loss: 2254.0716511034657
Epoch 17/15000, Loss: 2233.080515130357
Epoch 18/15000, Loss: 2213.161800939352
Epoch 19/15000, Loss: 2194.1778749954856
Epoch 20/15000, Loss: 2176.0133500955208
Epoch 21/15000, Loss: 2158.5714447627142
Epoch 22/15000, Loss: 2141.770401938724
Epoch 23/15000, Loss: 2125.540423044027
Epoch 24/15000, Loss: 2109.822834941112
Epoch 25/15000, Loss: 2094.56710

In [15]:
alphas = np.linspace(0.0001, 0.003, endpoint=True, num=10)
all_alphas: list[float] = []
epochs_amount_mean: list[int] = []
epochs_amount: list[int] = []
fig = go.Figure()
experiments_amount = 5
for alpha in alphas:
    epoch_sum = 0
    for i in range(experiments_amount):
        all_alphas.append(alpha)        
        dl = LRNN(n, 20, alpha)
        dl.train(train, 15000, learn_by_loss=True, max_loss=1500)
        epoch_sum += dl.epoch
        epochs_amount.append(dl.epoch)
    epochs_amount_mean.append(epoch_sum / experiments_amount)
fig.add_trace(go.Scatter(x=all_alphas, y=epochs_amount, mode='markers',
                         name='Все наблюдения'))
fig.add_trace(go.Scatter(x=alphas, y=epochs_amount_mean, mode='lines+markers',
                         name='Средние значения'))
fig.update_layout(title='Зависимость количества итераций обучения от коэффициента обучения',
                  xaxis_title='Значение коэффициента обучения',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

Epoch 1/15000, Loss: 3465.192465131964
Epoch 2/15000, Loss: 3098.831602509935
Epoch 3/15000, Loss: 2983.3277410557266
Epoch 4/15000, Loss: 2895.482097234072
Epoch 5/15000, Loss: 2820.179894751557
Epoch 6/15000, Loss: 2754.245318252815
Epoch 7/15000, Loss: 2695.7721060426165
Epoch 8/15000, Loss: 2643.373205096692
Epoch 9/15000, Loss: 2596.008508224201
Epoch 10/15000, Loss: 2552.8818552909506
Epoch 11/15000, Loss: 2513.3662594740454
Epoch 12/15000, Loss: 2476.9591415519467
Epoch 13/15000, Loss: 2443.2523048852436
Epoch 14/15000, Loss: 2411.904683864096
Epoch 15/15000, Loss: 2382.6320760870544
Epoch 16/15000, Loss: 2355.193511768252
Epoch 17/15000, Loss: 2329.383045466342
Epoch 18/15000, Loss: 2305.023921968711
Epoch 19/15000, Loss: 2281.962635733262
Epoch 20/15000, Loss: 2260.0671269557747
Epoch 21/15000, Loss: 2239.2212140197403
Epoch 22/15000, Loss: 2219.3224172064315
Epoch 23/15000, Loss: 2200.282192079696
Epoch 24/15000, Loss: 2182.0211313161794
Epoch 25/15000, Loss: 2164.46937652749

## Relationship between hidden layer dimension and epochs

In [16]:
p_s = (6, 8, 10, 12, 14, 16, 18, 20)
epochs_amount: list[int] = []
compression_rates: list[float] = []
for p in p_s:    
    dl = LRNN(n, p, 0.001)
    dl.train(train, 15000, learn_by_loss=True, max_loss=1900)
    epochs_amount.append(dl.epoch)

    compressed = compress_image(dl.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
    compression_info_size = (
        compressed.size * compressed.itemsize +
        network.W_dec.size * network.W_dec.itemsize +
        np.array(shape).size * np.array(shape).itemsize +
        np.array((block_height, block_width)).size * np.array((block_height, block_width)).itemsize 
    ) * 8
    compression_rates.append((img_array.size * img_array.itemsize * 8) / compression_info_size)

fig = go.Figure()
fig.add_trace(go.Scatter(x=compression_rates, y=epochs_amount, mode='lines+markers'))
fig.update_layout(title='Зависимость количества итераций обучения от коэффициента сжатия',
                  xaxis_title='Коэффициент сжатия',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

Epoch 1/15000, Loss: 2905.336631708075
Epoch 2/15000, Loss: 2447.1332908461727
Epoch 3/15000, Loss: 2284.25349906017
Epoch 4/15000, Loss: 2190.9310194517234
Epoch 5/15000, Loss: 2129.7326431857314
Epoch 6/15000, Loss: 2086.9878167441925
Epoch 7/15000, Loss: 2055.825687123712
Epoch 8/15000, Loss: 2032.3437950244195
Epoch 9/15000, Loss: 2014.1686277967362
Epoch 10/15000, Loss: 1999.7816995110911
Epoch 11/15000, Loss: 1988.168523058829
Epoch 12/15000, Loss: 1978.6290092480344
Epoch 13/15000, Loss: 1970.6648622646069
Epoch 14/15000, Loss: 1963.9152391139148
Epoch 15/15000, Loss: 1958.112702775272
Epoch 16/15000, Loss: 1953.0560447023947
Epoch 17/15000, Loss: 1948.5924615437903
Epoch 18/15000, Loss: 1944.604160679691
Epoch 19/15000, Loss: 1940.998816441792
Epoch 20/15000, Loss: 1937.7047570030998
Epoch 21/15000, Loss: 1934.6646076669717
Epoch 22/15000, Loss: 1931.8319539695385
Epoch 23/15000, Loss: 1929.1700005471669
Epoch 24/15000, Loss: 1926.6481109458834
Epoch 25/15000, Loss: 1924.241881

In [17]:
p_s = (6, 8, 10, 12, 14, 16, 18, 20)
all_ps: list[int] = []
all_compression_rates: list[float] = []
compression_rates: list[float] = []
epochs_amount_mean: list[int] = []
epochs_amount: list[int] = []
fig = go.Figure()
experiments_amount = 5
for p in p_s:
    epoch_sum = 0
    for i in range(experiments_amount):
        all_ps.append(p)        
        dl = LRNN(n, p, 0.001)
        dl.train(train, 15000, learn_by_loss=True, max_loss=1500)
        epoch_sum += dl.epoch
        epochs_amount.append(dl.epoch)

        compressed = compress_image(dl.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
        compression_info_size = (
            compressed.size * compressed.itemsize +
            network.W_dec.size * network.W_dec.itemsize +
            np.array(shape).size * np.array(shape).itemsize +
            np.array((block_height, block_width)).size * np.array((block_height, block_width)).itemsize 
        ) * 8
        all_compression_rates.append((img_array.size * img_array.itemsize * 8) / compression_info_size)
    compression_rates.append(all_compression_rates[-1])

    epochs_amount_mean.append(epoch_sum / experiments_amount)
fig.add_trace(go.Scatter(x=all_compression_rates, y=epochs_amount, mode='markers',
                         name='Все наблюдения'))
fig.add_trace(go.Scatter(x=compression_rates, y=epochs_amount_mean, mode='lines+markers',
                         name='Средние значения'))
fig.update_layout(title='Зависимость количества итераций обучения от коэффициента сжатия',
                  xaxis_title='Значение коэффичиента сжатия',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

Epoch 1/15000, Loss: 2887.751656245122
Epoch 2/15000, Loss: 2412.364638168766
Epoch 3/15000, Loss: 2256.873892526733
Epoch 4/15000, Loss: 2171.6807095244603
Epoch 5/15000, Loss: 2115.900977961093
Epoch 6/15000, Loss: 2076.452038274721
Epoch 7/15000, Loss: 2047.2822523657453
Epoch 8/15000, Loss: 2025.0335775253814
Epoch 9/15000, Loss: 2007.6488667218982
Epoch 10/15000, Loss: 1993.7869058139029
Epoch 11/15000, Loss: 1982.5343943117173
Epoch 12/15000, Loss: 1973.2493550669528
Epoch 13/15000, Loss: 1965.4681687628872
Epoch 14/15000, Loss: 1958.85196898779
Epoch 15/15000, Loss: 1953.1471601789353
Epoch 16/15000, Loss: 1948.1618637062963
Epoch 17/15000, Loss: 1943.749956781282
Epoch 18/15000, Loss: 1939.798019147388
Epoch 19/15000, Loss: 1936.2175609027222
Epoch 20/15000, Loss: 1932.9385973325534
Epoch 21/15000, Loss: 1929.9053491155773
Epoch 22/15000, Loss: 1927.073519704118
Epoch 23/15000, Loss: 1924.4069876351446
Epoch 24/15000, Loss: 1921.8764969204442
Epoch 25/15000, Loss: 1919.45746607

## Relationship between max error and epochs

In [21]:
max_errors = list(range(800, 3100, 100))
all_errors: list[float] = []
epochs_amount_mean: list[int] = []
epochs_amount: list[int] = []
fig = go.Figure()
experiments_amount = 5
for err in max_errors:
    epoch_sum = 0
    for i in range(experiments_amount):
        all_errors.append(err)        
        dl = LRNN(n, 50, 0.001)
        dl.train(train, 15000, learn_by_loss=True, max_loss=err)
        epoch_sum += dl.epoch
        epochs_amount.append(dl.epoch)
    epochs_amount_mean.append(epoch_sum / experiments_amount)
fig.add_trace(go.Scatter(x=all_errors, y=epochs_amount, mode='markers',
                         name='Все наблюдения'))
fig.add_trace(go.Scatter(x=max_errors, y=epochs_amount_mean, mode='lines+markers',
                         name='Средние значения'))
fig.update_layout(title='Зависимость количества итераций обучения от значения максимальной ошибки',
                  xaxis_title='Значение максимальной ошибки',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

Epoch 1/15000, Loss: 3258.0285555975
Epoch 2/15000, Loss: 2648.3577342916533
Epoch 3/15000, Loss: 2409.583635100408
Epoch 4/15000, Loss: 2248.2745616282
Epoch 5/15000, Loss: 2122.697562180633
Epoch 6/15000, Loss: 2020.3316150338603
Epoch 7/15000, Loss: 1934.8626048627657
Epoch 8/15000, Loss: 1862.1101399363977
Epoch 9/15000, Loss: 1799.0389821320127
Epoch 10/15000, Loss: 1743.410807826286
Epoch 11/15000, Loss: 1693.5836961396762
Epoch 12/15000, Loss: 1648.3610493337428
Epoch 13/15000, Loss: 1606.8739035203046
Epoch 14/15000, Loss: 1568.4878993518987
Epoch 15/15000, Loss: 1532.7352427141025
Epoch 16/15000, Loss: 1499.2661801410347
Epoch 17/15000, Loss: 1467.812532045019
Epoch 18/15000, Loss: 1438.1643689708544
Epoch 19/15000, Loss: 1410.152040018905
Epoch 20/15000, Loss: 1383.6352298697502
Epoch 21/15000, Loss: 1358.4937347711007
Epoch 22/15000, Loss: 1334.6234260158944
Epoch 23/15000, Loss: 1311.9315975862964
Epoch 24/15000, Loss: 1290.334807805732
Epoch 25/15000, Loss: 1269.7571672602

## Relationship between image and epochs

In [19]:
# Collecting everything

block_width = 10
block_height = 10

n = block_height * block_width
# Hidden layer neuron amount
p = 50 

img = Image.open('images/winter.jpg')
img_array = np.asarray(img)
shape = img_array.shape
blocks = image_to_blocks(img_array, block_height, block_width, overlap=0)

l = len(blocks)
# Compression coeff
print('Z =', (n*l) / ((n+l) * p+2))

color_df = ((2 * blocks / MAX_RGB_VALUE) - 1).reshape(len(blocks), -1, 3).transpose(0, 2, 1).reshape(-1, block_height * block_width)
train = np.matrix(color_df[np.random.choice(color_df.shape[0], int(color_df.shape[0] * 0.05))])

network = LRNN(n, p, 0.001)
network.train(train, 15000, learn_by_loss=True, max_loss=1500)

print(network.epoch)

# compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
# dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
# dimg_array = np.asarray(dimg)
# dimg.save('compression-decompression_test.jpg')

Z = 1.9230473377332657
Epoch 1/15000, Loss: 3481.4024117730783
Epoch 2/15000, Loss: 2774.7320125241795
Epoch 3/15000, Loss: 2470.901055101439
Epoch 4/15000, Loss: 2266.449808752876
Epoch 5/15000, Loss: 2120.060684484284
Epoch 6/15000, Loss: 2011.7924062138106
Epoch 7/15000, Loss: 1927.7975495185017
Epoch 8/15000, Loss: 1859.1340562749253
Epoch 9/15000, Loss: 1800.4643214933894
Epoch 10/15000, Loss: 1748.6834147447132
Epoch 11/15000, Loss: 1701.9595116666844
Epoch 12/15000, Loss: 1659.1689185418895
Epoch 13/15000, Loss: 1619.584635850346
Epoch 14/15000, Loss: 1582.707319877678
Epoch 15/15000, Loss: 1548.1737423189174
Epoch 16/15000, Loss: 1515.7059922597807
Epoch 17/15000, Loss: 1485.0824910454144
17


In [26]:
np.dot([-3, -2, -2, -3, -1, 2], [0, 0, 0, 0, -1, -2])

np.int64(-3)

In [35]:
0.46 * (1 - 0.8) * -0.9

-0.08279999999999998